In [5]:
import os
from collections import Counter
import re
import string
import numpy as np
import random
import itertools

with open('./dat410_europarl/europarl-v7.fr-en.lc.en','r',encoding='utf-8') as f:
    words = f.read()
    words = re.split('[^\w+]',words)
words = [x for x in words if x]

In [3]:
#read files
#english
de_en_en = open('./dat410_europarl/europarl-v7.de-en.lc.en','r', encoding = "utf-8").read()
fr_en_en = open('./dat410_europarl/europarl-v7.fr-en.lc.en','r',encoding = "utf-8").read()
sv_en_en = open('./dat410_europarl/europarl-v7.sv-en.lc.en','r',encoding = "utf-8").read()
all_en = de_en_en + " " + fr_en_en + " " + sv_en_en + " "

#other language
de_en_de = open('./dat410_europarl/europarl-v7.de-en.lc.de','r',encoding = "utf-8").read()
fe_en_fr = open('./dat410_europarl/europarl-v7.fr-en.lc.fr','r',encoding = "utf-8").read()
sv_en_sv = open('./dat410_europarl/europarl-v7.sv-en.lc.sv','r',encoding = "utf-8").read()
all_other = de_en_de + " " + fe_en_fr + " " + sv_en_sv + " "

#removing special symbols 
def clean_file(name):
    punctuation = string.punctuation
    for i in punctuation:
        name = name.replace(i,'')  
    return name
#removing special symbols 
def clean_file2(textname):
    xxx = re.sub(r'[^\w]', ' ',textname)
    return xxx
    
#find most frequent words
def most_frequent_words(file_name, num):
    #file = file_name.replace(",","")
    #file = file.replace(".","")
    words = Counter(file_name.split()).most_common(num)
    print(words)
#Germany
de_clean = clean_file(de_en_de)
de = most_frequent_words(de_clean,10)
#French
fr_clean = clean_file(fe_en_fr)
fr = most_frequent_words(fr_clean,10)

[('die', 10521), ('der', 9374), ('und', 7028), ('in', 4175), ('zu', 3169), ('den', 2976), ('wir', 2863), ('daß', 2738), ('ich', 2670), ('das', 2669)]
[('apos', 16729), ('de', 14520), ('la', 9746), ('et', 6619), ('l', 6536), ('le', 6174), ('les', 5585), ('à', 5500), ('des', 5232), ('que', 4797)]


In [4]:
 # probability of 'speaker' and 'zebra'
all_texts = all_en + " " + all_other + " "
all_texts_clean = clean_file(all_texts)
all_texts_counters = Counter(all_texts_clean.split())
speaker = all_texts_counters["speaker"]/sum(all_texts_counters.values())
zebra = all_texts_counters["zebra"]/sum(all_texts_counters.values())
print(f"Probability of speaker: {speaker}")
print(f"Probability of zebra: {zebra}")

Probability of speaker: 2.1350917863018982e-05
Probability of zebra: 0.0


In [29]:
def find_bigram(lang):
    ''''
    find the bigram and unigram and count how many times for each to appear in the English text
    ''''
    bigram = []
    unigram = []
    for i in range(len(lang)-1):
        bigram.append((lang[i],lang[i+1]))
    for i in range(len(lang)):
        unigram.append(lang[i])
        
    count_bigram = dict(Counter(bigram))
    count_unigram = dict(Counter(unigram))
    return bigram, count_bigram, count_unigram # bigram(list),  count_bigram(dict),  count_unigram(dict)

def probability(sentence, lang):
    '''
    calculate the probability of each bigram and unigram of one individual input sentence
    '''
    bigrams, count_bigrams, count_unigram = find_bigram(lang)
    pro_bigram = {} # dict to store the probability of each bigram
    sentence_split = sentence.split()
    bi_sen = [] # list to store bigrams of the input sentence
    prob = 1
    for bigram in list(count_bigrams.keys()):
        pro_bigram[bigram] = count_bigrams[bigram]/count_unigram[bigram[0]]
        
    for i in range(len(sentence_split)-1):
        bi_sen.append((sentence_split[i],sentence_split[i+1]))
        if bi_sen[i] in pro_bigram:
            prob *= pro_bigram[bi_sen[i]]
        else:
            # if the bigram does not exist in the bigrams of the English text, we consider it appearing 0.1 times
            prob *= 0.1/sum(count_bigrams.values())
    return prob



In [37]:
# load the English file and French file
En_file = open('./dat410_europarl/europarl-v7.fr-en.lc.en',encoding = 'utf-8').read()
En_file = En_file.splitlines()
Fr_file = open('./dat410_europarl/europarl-v7.fr-en.lc.fr',encoding = 'utf-8').read()
Fr_file = Fr_file.splitlines()
En_file
sentence_pair = [] # list to store parallel sentence pair
for i in range(len(En_file)):
    sentence_pair.append([Fr_file[i],En_file[i]])
word_pair = []    
# split each sentence pair into individual word
for i in range(len( sentence_pair)):
    En_word = re.split('[^\w+]', sentence_pair[i][1])
    En_word = [x for x in En_word if x]
    En_word.append('null')
    
    Fr_word = re.split('[^\w+]', sentence_pair[i][0])
    Fr_word = [x for x in Fr_word if x]
    
    word_pair.append([Fr_word,En_word])

In [38]:
def EM_ENtoFR(integ, word_pair):
'''
translation model to find the alignment
'''
    # initialization
    trans = {}
    for i in range(len(word_pair)):
        for en_word in word_pair[i][1]:
            for fr_word in word_pair[i][0]:
                trans[en_word,fr_word] = random.random()

    trans_total = {}
    delta = {}
    for t in range(integ):
        soft_cnt = {}
        soft_cntuni = {}
        for k in range(len(word_pair)):
            for fr_word in word_pair[k][0]:
                soft_cntuni[fr_word] = 0
                for en_word in word_pair[k][1]:
                    soft_cnt[(en_word,fr_word)] = 0

        for k in range(len(word_pair)):
            for en_word in word_pair[k][1]:
                trans_total[en_word] = 0
                for fr_word in word_pair[k][0]:
                    trans_total[en_word]+= trans[(en_word,fr_word)]
                for fr_word in word_pair[k][0]:
                    delta[(en_word,fr_word)] = trans[(en_word,fr_word)]/trans_total[en_word]
                    soft_cnt[(en_word,fr_word)] +=  delta[(en_word,fr_word)]
                    soft_cntuni[fr_word] += delta[(en_word,fr_word)]

        for k in range(len(word_pair)):
            for en_word in word_pair[k][1]:
                for fr_word in word_pair[k][0]:
                    trans[(en_word,fr_word)] = soft_cnt[(en_word,fr_word)]/soft_cntuni[fr_word]
    return trans

In [15]:
def trans_ENword(word,t,word_pair):
    '''
    function to find the most possible 10 words in French given an English word
    '''
    trans = EM_ENtoFR(t,word_pair)
    word_list = {}
    word_topten = []
    for pair in trans.keys():
        if pair[0] == word:
            word_list[pair[1]] = trans[pair]
    word_top = sorted(word_list.items(), key = lambda x:x[1], reverse = True)
    
    for i in range(10):
        word_topten.append(word_top[i][0])
    
    return word_topten

trans_ENword('european',3,word_pair)
   
    
    

['européennes',
 'européen',
 'européens',
 'union',
 'européenne',
 'risquées',
 'prison',
 'pic',
 'labour',
 'aérienne']

In [16]:
def trans_FRword(word,t,word_pair):
    '''
    find the most possible English word given a French word
    '''
    trans = EM_ENtoFR(t,word_pair)
    word_list = {}
    for pair in trans.keys():
        if pair[1] == word:
            word_list[pair[0]] = trans[pair]
    word_top = sorted(word_list.items(), key = lambda x:x[1], reverse = True)
    word_top = word_top[0][0]
    
    return word_top


In [45]:
def trans_sentence(sentence, t, word_pair, words):
    '''
    translate the given French sentence
    '''
    sen_trans = [] # store the words in the input sentence
    sen_split = sentence.split() # split the sentence
    for word in sen_split:
        sen_trans.append(trans_FRword(word,t,word_pair))
    if 'null' in sen_trans:
        # remove "null" in the word list
        sen_trans = sen_trans.remove('null')
    tmp_list = itertools.permutations(sen_trans)
    permu_word = [] # store the permutations of the given word list
    for res in tmp_list:
        permu_word.append(list(res))
    
    for sub_sen in permu_word:
        # find in what sequence the sentence will have the highest probability
        sub_sen = ' '.join(sub_sen)
        sen_permu[sub_sen] = probability(sub_sen,words)
    sen_top = sorted(sen_permu.items(), key = lambda x:x[1], reverse = True)
    return sen_top[0][0]
trans_sentence('une pétition a déjà été introduite',20,word_pair,words)

'people has already been introduced a'

In [46]:
trans_sentence('je suis un étudiant',10,word_pair,words)

'i am a darker'